In [13]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [14]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [15]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [21]:
data18 = data.loc[data['Year']== '2018']
data18

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [26]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data18['content']))

print(data_words[:1])

[['global', 'bass', 'the', 'internet', 'centric', 'underground', 'scene', 'that', 'began', 'flourishing', 'around', 'the', 'world', 'in', 'the', 'late', 'emerged', 'when', 'young', 'producers', 'took', 'their', 'countries', 'folk', 'traditions', 'and', 'updated', 'them', 'for', 'millennial', 'dancefloor', 'they', 'used', 'new', 'technology', 'to', 'connect', 'previously', 'collapsed', 'musical', 'boundaries', 'cumbia', 'became', 'digitized', 'colombia', 'mexico', 'salsa', 'got', 'remixed', 'puerto', 'rico', 'new', 'york', 'canada', 'and', 'some', 'even', 'transmogrified', 'andean', 'pan', 'flutes', 'into', 'intellectual', 'bangers', 'global', 'bass', 'became', 'precursor', 'to', 'the', 'increasingly', 'boundless', 'way', 'pop', 'music', 'works', 'now', 'even', 'as', 'it', 'became', 'flattened', 'into', 'limp', 'and', 'canned', 'versions', 'of', 'what', 'some', 'people', 'dreadfully', 'termed', 'trop', 'house', 'not', 'to', 'mention', 'the', 'overall', 'poisonous', 'creep', 'of', 'edm',

In [27]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['global', 'bass', 'the', 'internet', 'centric', 'underground', 'scene', 'that', 'began', 'flourishing', 'around', 'the', 'world', 'in', 'the', 'late', 'emerged', 'when', 'young', 'producers', 'took', 'their', 'countries', 'folk', 'traditions', 'and', 'updated', 'them', 'for', 'millennial', 'dancefloor', 'they', 'used', 'new', 'technology', 'to', 'connect', 'previously', 'collapsed', 'musical', 'boundaries', 'cumbia', 'became', 'digitized', 'colombia', 'mexico', 'salsa', 'got', 'remixed', 'puerto', 'rico', 'new_york', 'canada', 'and', 'some', 'even', 'transmogrified', 'andean', 'pan', 'flutes', 'into', 'intellectual', 'bangers', 'global', 'bass', 'became', 'precursor', 'to', 'the', 'increasingly', 'boundless', 'way', 'pop', 'music', 'works', 'now', 'even', 'as', 'it', 'became', 'flattened', 'into', 'limp', 'and', 'canned', 'versions', 'of', 'what', 'some', 'people', 'dreadfully', 'termed', 'trop_house', 'not', 'to', 'mention', 'the', 'overall', 'poisonous', 'creep', 'of', 'edm', 'into'

In [28]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [30]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['global', 'bass', 'internet', 'centric', 'underground', 'scene', 'begin', 'flourish', 'world', 'late', 'emerge', 'young', 'producer', 'take', 'country', 'folk', 'tradition', 'update', 'millennial', 'dancefloor', 'use', 'new', 'technology', 'connect', 'previously', 'collapse', 'musical', 'boundary', 'cumbia', 'become', 'digitize', 'mexico', 'salsa', 'get', 'remix', 'puerto', 'rico', 'new_york', 'canada', 'even', 'transmogrify', 'andean', 'pan', 'flute', 'intellectual', 'banger', 'global', 'bass', 'become', 'precursor', 'increasingly', 'boundless', 'way', 'pop', 'music', 'work', 'even', 'become', 'flatten', 'limp', 'canned', 'version', 'people', 'dreadfully', 'term', 'trop_house', 'mention', 'overall', 'poisonous', 'creep', 'edm', 'bloodstream', 'concept', 'still', 'intriguing', 'internet', 'homogenize', 'individual', 'music', 'culture', 'big', 'studio', 'quantize', 'mish', 'mash', 'musician', 'retain', 'singularity', 'hyper', 'local', 'often', 'dissipate', 'history', 'culture', 'alter

In [38]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#filter extreme values in dictionary
id2word.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 3), (74, 1), (75, 1), (76, 1), (77, 3), (78, 1), (79, 1), (80, 1), (81, 2), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 2), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 1)

In [39]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [40]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.005*"verse" + 0.004*"hook" + 0.004*"talk" + 0.003*"soul" + '
  '0.003*"produce" + 0.003*"fuck" + 0.003*"boy" + 0.003*"bad" + 0.003*"street" '
  '+ 0.003*"friend"'),
 (1,
  '0.037*"film" + 0.021*"jazz" + 0.019*"performance" + 0.016*"original" + '
  '0.015*"version" + 0.015*"score" + 0.014*"cover" + 0.014*"soundtrack" + '
  '0.014*"session" + 0.013*"recording"'),
 (2,
  '0.098*"prince" + 0.067*"feminist" + 0.035*"hurricane" + 0.030*"cook" + '
  '0.008*"reggae" + 0.007*"resilience" + 0.000*"purple" + 0.000*"unreleased" + '
  '0.000*"artifact" + 0.000*"seize"'),
 (3,
  '0.146*"metal" + 0.084*"hardcore" + 0.043*"thrash" + 0.043*"doom" + '
  '0.039*"riff" + 0.038*"black_metal" + 0.027*"punk" + 0.019*"industrial" + '
  '0.018*"extreme" + 0.016*"noise"'),
 (4,
  '0.060*"logic" + 0.057*"bird" + 0.036*"richard" + 0.032*"cruise" + '
  '0.030*"otherworldly" + 0.030*"field_recording" + 0.029*"ambient" + '
  '0.029*"equal" + 0.027*"exquisite" + 0.026*"journalist"'),
 (5,
  '0.007*"sample" 

In [41]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.086771682154495

Coherence Score:  0.4235866742486105


In [42]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.009883  0.051833       1        1  35.093983
0     -0.044011  0.086213       2        1  28.617657
5      0.056730  0.075098       3        1  18.693682
7     -0.149319  0.202254       4        1  13.454962
1     -0.329764 -0.202407       5        1   2.612627
9      0.152977 -0.240177       6        1   0.692431
3      0.100067 -0.015660       7        1   0.436628
4      0.105500 -0.015303       8        1   0.273444
2      0.059325  0.029884       9        1   0.067033
6      0.058378  0.028265      10        1   0.057555, topic_info=     Category        Freq           Term       Total  loglift  logprob
267   Default  337.000000           punk  337.000000  30.0000  30.0000
1299  Default  271.000000           trap  271.000000  29.0000  29.0000
633   Default  257.000000           riff  257.000000  28.0000  28.0000
957   Default  244.000000        ambient  244.000000  27.0000  27.0000
1382  Default  291.000000          noise  291.000000  26.0000  26.0000
1238  Default  258.000000           flow  258.000000  25.0000  25.0000
806   Default  345.000000           jazz  345.000000  24.0000  24.0000
1520  Default  263.000000          heavy  263.000000  23.0000  23.0000
918   Default  223.000000           tape  223.000000  22.0000  22.0000
355   Default  200.000000      guitarist  200.000000  21.0000  21.0000
770   Default  235.000000           film  235.000000  20.0000  20.0000
252   Default  147.000000          metal  147.000000  19.0000  19.0000
842   Default  261.000000    performance  261.000000  18.0000  18.0000
729   Default  338.000000          cover  338.000000  17.0000  17.0000
1387  Default  263.000000       original  263.000000  16.0000  16.0000
171   Default  442.000000          woman  442.000000  15.0000  15.0000
167   Default  236.000000        version  236.000000  14.0000  14.0000
1995  Default  128.000000        mixtape  128.000000  13.0000  13.0000
342   Default  171.000000        drummer  171.000000  12.0000  12.0000
1317  Default  261.000000           blue  261.000000  11.0000  11.0000
1064  Default  440.000000          black  440.000000  10.0000  10.0000
316   Default  214.000000       audience  214.000000   9.0000   9.0000
33    Default  278.000000        country  278.000000   8.0000   8.0000
1786  Default  186.000000      recording  186.000000   7.0000   7.0000
1639  Default  252.000000           tune  252.000000   6.0000   6.0000
901   Default  190.000000     soundtrack  190.000000   5.0000   5.0000
1144  Default  357.000000          verse  357.000000   4.0000   4.0000
331   Default  262.000000          death  262.000000   3.0000   3.0000
926   Default  207.000000           tour  207.000000   2.0000   2.0000
982   Default  205.000000           hour  205.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
2347  Topic10    0.024903         vacuum   11.367526   1.3367  -8.5797
1509  Topic10    0.024905    charismatic   11.370336   1.3365  -8.5796
3493  Topic10    0.024901         formal   11.385784   1.3350  -8.5798
2229  Topic10    0.024910          horny   11.412724   1.3330  -8.5794
3355  Topic10    0.024900  differentiate   11.456231   1.3288  -8.5798
768   Topic10    0.024905       february   11.594125   1.3170  -8.5796
2595  Topic10    0.024913     invocation   22.474806   0.6554  -8.5793
2081  Topic10    0.024913        torment   17.904282   0.8828  -8.5793
3553  Topic10    0.024912     incredibly   17.739153   0.8920  -8.5793
1613  Topic10    0.024912        radiant   29.915413   0.3694  -8.5793
3247  Topic10    0.024912  communication   16.735256   0.9502  -8.5793
34    Topic10    0.024911     creativity   20.525568   0.7461  -8.5793
2903  Topic10    0.024911     complexity   54.161453  -0.2242  -8.5793
1034  Topic10    0.024911         stroll   21.597933   0.6951  -8.5794
2133  Topic10    0.024910         reduce   